In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek


from typing import Tuple

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

Data Preprocessing

In [2]:
#import dataset 
pbc_data = pd.read_csv(r'pbc.csv')
pbc_data 

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,1,400,D,D-penicillamine,21464,F,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,2,4500,C,D-penicillamine,20617,F,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,3,1012,D,D-penicillamine,25594,M,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,4,1925,D,D-penicillamine,19994,F,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
4,5,1504,CL,Placebo,13918,F,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,414,681,D,NaN,24472,F,NaN,NaN,NaN,N,1.2,NaN,2.96,NaN,NaN,NaN,NaN,174.0,10.9,3.0
414,415,1103,C,NaN,14245,F,NaN,NaN,NaN,N,0.9,NaN,3.83,NaN,NaN,NaN,NaN,180.0,11.2,4.0
415,416,1055,C,NaN,20819,F,NaN,NaN,NaN,N,1.6,NaN,3.42,NaN,NaN,NaN,NaN,143.0,9.9,3.0
416,417,691,C,NaN,21185,F,NaN,NaN,NaN,N,0.8,NaN,3.75,NaN,NaN,NaN,NaN,269.0,10.4,3.0


In [3]:
#remove features which are not relevant in determine the stage
pbc_data = pbc_data.drop(["ID"], axis=1) 
pbc_data = pbc_data.drop(["N_Days"], axis=1)
pbc_data = pbc_data.drop(["Drug"], axis=1) 
pbc_data = pbc_data.drop(["Status"], axis=1)
pbc_data 

,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,21464,F,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,20617,F,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,25594,M,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,19994,F,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
4,13918,F,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,24472,F,NaN,NaN,NaN,N,1.2,NaN,2.96,NaN,NaN,NaN,NaN,174.0,10.9,3.0
414,14245,F,NaN,NaN,NaN,N,0.9,NaN,3.83,NaN,NaN,NaN,NaN,180.0,11.2,4.0
415,20819,F,NaN,NaN,NaN,N,1.6,NaN,3.42,NaN,NaN,NaN,NaN,143.0,9.9,3.0
416,21185,F,NaN,NaN,NaN,N,0.8,NaN,3.75,NaN,NaN,NaN,NaN,269.0,10.4,3.0


In [4]:
# datatypes
pbc_data.dtypes

Age                int64
Sex               object
Ascites           object
Hepatomegaly      object
Spiders           object
Edema             object
Bilirubin        float64
Cholesterol      float64
Albumin          float64
Copper           float64
Alk_Phos         float64
SGOT             float64
Tryglicerides    float64
Platelets        float64
Prothrombin      float64
Stage            float64
dtype: object

In [5]:
# combine into two classes
pbc_data["Stage"]=pbc_data["Stage"].replace([3.0, 2.0, 1.0], 0.0)
pbc_data["Stage"]=pbc_data["Stage"].replace([4.0], 1.0)

In [6]:
# separate categorical features and numerical features
categorical_features = []
numerical_features = []
for i in pbc_data.columns:
    if pbc_data[i].nunique() > 3:
        numerical_features.append(i)
    else:
        categorical_features.append(i)
print("Categorical features: ", categorical_features)
print("Numerical features: ", numerical_features)

Categorical features:  ['Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Stage']
Numerical features:  ['Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']


In [7]:
# lable encoding
pbc_data[categorical_features] = pbc_data [categorical_features].apply(lambda series: pd.Series(LabelEncoder().fit_transform(series[series.notnull()]), index= series[series.notnull()].index))
pbc_data

,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,21464,0,1.0,1.0,1.0,2,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,1.0
1,20617,0,0.0,1.0,1.0,0,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,0.0
2,25594,1,0.0,0.0,0.0,1,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,1.0
3,19994,0,0.0,1.0,1.0,1,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,1.0
4,13918,0,0.0,1.0,1.0,0,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,24472,0,NaN,NaN,NaN,0,1.2,NaN,2.96,NaN,NaN,NaN,NaN,174.0,10.9,0.0
414,14245,0,NaN,NaN,NaN,0,0.9,NaN,3.83,NaN,NaN,NaN,NaN,180.0,11.2,1.0
415,20819,0,NaN,NaN,NaN,0,1.6,NaN,3.42,NaN,NaN,NaN,NaN,143.0,9.9,0.0
416,21185,0,NaN,NaN,NaN,0,0.8,NaN,3.75,NaN,NaN,NaN,NaN,269.0,10.4,0.0


In [8]:
# normalization
target = pbc_data["Stage"]
features = pbc_data.drop(["Stage"], axis = 1)
scaler = MinMaxScaler()
pbc_data_normalized = scaler.fit_transform(features)
pbc_data_normalized = pd.DataFrame(data = pbc_data_normalized, columns = features.columns)
pbc_data_normalized = pbc_data_normalized.join(target)

pbc_data_normalized

,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,0.622822,0.0,1.0,1.0,1.0,1.0,0.512635,0.085196,0.238806,0.260274,0.105279,0.258993,0.246018,0.194234,0.355556,1.0
1,0.578364,0.0,0.0,1.0,1.0,0.0,0.028881,0.109970,0.813433,0.085616,0.523509,0.202298,0.097345,0.241275,0.177778,0.0
2,0.839597,1.0,0.0,0.0,0.0,0.5,0.039711,0.033837,0.567164,0.352740,0.016724,0.161871,0.038938,0.135053,0.333333,1.0
3,0.545664,0.0,0.0,1.0,1.0,0.5,0.054152,0.074924,0.216418,0.102740,0.429723,0.079554,0.104425,0.183612,0.144444,1.0
4,0.226748,0.0,0.0,1.0,1.0,0.0,0.111913,0.096073,0.585821,0.238014,0.028143,0.201439,0.069027,0.112291,0.211111,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.780705,0.0,NaN,NaN,NaN,0.0,0.032491,NaN,0.373134,NaN,NaN,NaN,NaN,0.169954,0.211111,0.0
414,0.243911,0.0,NaN,NaN,NaN,0.0,0.021661,NaN,0.697761,NaN,NaN,NaN,NaN,0.179059,0.244444,1.0
415,0.588967,0.0,NaN,NaN,NaN,0.0,0.046931,NaN,0.544776,NaN,NaN,NaN,NaN,0.122914,0.100000,0.0
416,0.608178,0.0,NaN,NaN,NaN,0.0,0.018051,NaN,0.667910,NaN,NaN,NaN,NaN,0.314112,0.155556,0.0


In [9]:
# drop the whole row where the stage is empyty
pbc_data_normalized.dropna(subset=["Stage"], inplace = True)
pbc_data_normalized

,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,0.622822,0.0,1.0,1.0,1.0,1.0,0.512635,0.085196,0.238806,0.260274,0.105279,0.258993,0.246018,0.194234,0.355556,1.0
1,0.578364,0.0,0.0,1.0,1.0,0.0,0.028881,0.109970,0.813433,0.085616,0.523509,0.202298,0.097345,0.241275,0.177778,0.0
2,0.839597,1.0,0.0,0.0,0.0,0.5,0.039711,0.033837,0.567164,0.352740,0.016724,0.161871,0.038938,0.135053,0.333333,1.0
3,0.545664,0.0,0.0,1.0,1.0,0.5,0.054152,0.074924,0.216418,0.102740,0.429723,0.079554,0.104425,0.183612,0.144444,1.0
4,0.226748,0.0,0.0,1.0,1.0,0.0,0.111913,0.096073,0.585821,0.238014,0.028143,0.201439,0.069027,0.112291,0.211111,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.780705,0.0,NaN,NaN,NaN,0.0,0.032491,NaN,0.373134,NaN,NaN,NaN,NaN,0.169954,0.211111,0.0
414,0.243911,0.0,NaN,NaN,NaN,0.0,0.021661,NaN,0.697761,NaN,NaN,NaN,NaN,0.179059,0.244444,1.0
415,0.588967,0.0,NaN,NaN,NaN,0.0,0.046931,NaN,0.544776,NaN,NaN,NaN,NaN,0.122914,0.100000,0.0
416,0.608178,0.0,NaN,NaN,NaN,0.0,0.018051,NaN,0.667910,NaN,NaN,NaN,NaN,0.314112,0.155556,0.0


In [10]:
# number of null data
pbc_data_normalized.isnull().sum()

Age                0
Sex                0
Ascites          100
Hepatomegaly     100
Spiders          100
Edema              0
Bilirubin          0
Cholesterol      128
Albumin            0
Copper           102
Alk_Phos         100
SGOT             100
Tryglicerides    130
Platelets         11
Prothrombin        2
Stage              0
dtype: int64

In [11]:
# imputation
imputer = KNNImputer(n_neighbors = 10)
pbc_data_impute = pd.DataFrame(imputer.fit_transform(pbc_data_normalized), columns = pbc_data_normalized.columns)
c1 = ['Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Stage']
pbc_data_impute[c1] = pbc_data_impute[c1].values.round(0)
pbc_data_impute[c1] = pbc_data_impute[c1].astype(int)

pbc_data_impute

,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,0.622822,0,1,1,1,1.0,0.512635,0.085196,0.238806,0.260274,0.105279,0.258993,0.246018,0.194234,0.355556,1
1,0.578364,0,0,1,1,0.0,0.028881,0.109970,0.813433,0.085616,0.523509,0.202298,0.097345,0.241275,0.177778,0
2,0.839597,1,0,0,0,0.5,0.039711,0.033837,0.567164,0.352740,0.016724,0.161871,0.038938,0.135053,0.333333,1
3,0.545664,0,0,1,1,0.5,0.054152,0.074924,0.216418,0.102740,0.429723,0.079554,0.104425,0.183612,0.144444,1
4,0.226748,0,0,1,1,0.0,0.111913,0.096073,0.585821,0.238014,0.028143,0.201439,0.069027,0.112291,0.211111,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,0.780705,0,0,1,0,0.0,0.032491,0.157885,0.373134,0.097260,0.081122,0.220724,0.156637,0.169954,0.211111,0
408,0.243911,0,0,1,0,0.0,0.021661,0.125982,0.697761,0.107021,0.089491,0.219691,0.092035,0.179059,0.244444,1
409,0.588967,0,0,0,0,0.0,0.046931,0.125680,0.544776,0.071747,0.056345,0.184312,0.125841,0.122914,0.100000,0
410,0.608178,0,0,0,0,0.0,0.018051,0.083625,0.667910,0.053082,0.038656,0.125551,0.117522,0.314112,0.155556,0


In [12]:
y = pbc_data_impute["Stage"]
X = pbc_data_impute.drop(["Stage"], axis=1)

In [13]:
# datatypes
pbc_data_impute.dtypes

Age              float64
Sex                int32
Ascites            int32
Hepatomegaly       int32
Spiders            int32
Edema            float64
Bilirubin        float64
Cholesterol      float64
Albumin          float64
Copper           float64
Alk_Phos         float64
SGOT             float64
Tryglicerides    float64
Platelets        float64
Prothrombin      float64
Stage              int32
dtype: object

In [14]:
# train test split, 30% test, 70% train
y = pbc_data_impute["Stage"]
X = pbc_data_impute.drop(["Stage"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1)

In [15]:
print(y_train.tolist().count(0))
print(y_train.tolist().count(1))

217
112


In [16]:
resample = SMOTE(random_state = 1)
X_train, y_train = resample.fit_resample(X_train, y_train)
print(y_train.tolist().count(0))
print(y_train.tolist().count(1))

217
217


In [17]:
print(y_test.tolist().count(0))
print(y_test.tolist().count(1))

51
32


Classification without feature selection

In [18]:
# random forest
model = RandomForestClassifier(random_state = 1)
train_model = model.fit(X_train, y_train)
pred_test = train_model.predict(X_test)

print("Accuracy: {}".format((accuracy_score(pred_test, y_test)*100).round(2)))
print("Precision: {}".format((precision_score(pred_test, y_test)*100).round(2)))
print("Recall: {}".format((recall_score(pred_test, y_test)*100).round(2)))
print("F1-score: {}".format((f1_score(pred_test, y_test)*100).round(2)))
print("Confusion matrix: ", confusion_matrix(pred_test, y_test))

Accuracy: 75.9
Precision: 75.0
Recall: 66.67
F1-score: 70.59
Confusion matrix:  [[39  8]
 [12 24]]


In [19]:
# scm
model = SVC(random_state = 1)
train_model = model.fit(X_train, y_train)
pred_test = train_model.predict(X_test)

print("Accuracy: {}".format((accuracy_score(pred_test, y_test)*100).round(2)))
print("Precision: {}".format((precision_score(pred_test, y_test)*100).round(2)))
print("Recall: {}".format((recall_score(pred_test, y_test)*100).round(2)))
print("F1-score: {}".format((f1_score(pred_test, y_test)*100).round(2)))
print("Confusion matrix: ", confusion_matrix(pred_test, y_test))

Accuracy: 72.29
Precision: 81.25
Recall: 60.47
F1-score: 69.33
Confusion matrix:  [[34  6]
 [17 26]]


In [20]:
# xgboost
model = XGBClassifier(random_state = 1)
train_model = model.fit(X_train, y_train)
pred_test = train_model.predict(X_test)

print("Accuracy: {}".format((accuracy_score(pred_test, y_test)*100).round(2)))
print("Precision: {}".format((precision_score(pred_test, y_test)*100).round(2)))
print("Recall: {}".format((recall_score(pred_test, y_test)*100).round(2)))
print("F1-score: {}".format((f1_score(pred_test, y_test)*100).round(2)))
print("Confusion matrix: ", confusion_matrix(pred_test, y_test))

Accuracy: 71.08
Precision: 75.0
Recall: 60.0
F1-score: 66.67
Confusion matrix:  [[35  8]
 [16 24]]


Ensemble Feature Selection

In [21]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from skfeature.function.similarity_based import fisher_score
from skfeature.function.similarity_based import reliefF

#feature selection
num_feat = 3
c2 = SelectKBest(score_func=chi2, k=num_feat).fit(X, y)
c2_feat = X.columns[c2.get_support()]
c2_selected = X[c2_feat]
print("Chi2: ", c2_selected.columns)

mi = SelectKBest(score_func=mutual_info_classif, k=num_feat).fit(X, y)
mi_feat = X.columns[mi.get_support()]
mi_selected = X[mi_feat]
print("Mutual information: ", mi_selected.columns)

fs = fisher_score.fisher_score(np.array(X), np.array(y))
fs_idx = np.argsort(fs, 0)
fs_arr = fs_idx[::-1]
fs_feat = fs_arr[:num_feat]
fs_selected = X.iloc[:,np.r_[fs_feat]]
print("Fisher Score: ", fs_selected.columns)

rf = reliefF.reliefF(np.array(X), np.array(y))
rf_idx = np.argsort(rf, 0)
rf_arr = rf_idx[::-1]
rf_feat = rf_arr[:num_feat]
rf_selected = X.iloc[:,np.r_[rf_feat]]
print("ReliefF: ", rf_selected.columns)

union_set = list(set().union(c2_selected.columns, mi_selected.columns, fs_selected.columns, rf_selected.columns))
num_of_union_feature = len(union_set)
print("Union Set: ", union_set)
print("Number of features: " + str(num_of_union_feature))

Chi2:  Index(['Ascites', 'Hepatomegaly', 'Spiders'], dtype='object')
Mutual information:  Index(['Hepatomegaly', 'Alk_Phos', 'Prothrombin'], dtype='object')
Fisher Score:  Index(['Hepatomegaly', 'Albumin', 'Spiders'], dtype='object')
ReliefF:  Index(['Age', 'Ascites', 'Cholesterol'], dtype='object')
Union Set:  ['Hepatomegaly', 'Ascites', 'Prothrombin', 'Albumin', 'Spiders', 'Cholesterol', 'Alk_Phos', 'Age']
Number of features: 8


In [22]:
# union feature
ensemble_fs = X[union_set]
ensemble_fs

,Hepatomegaly,Ascites,Prothrombin,Albumin,Spiders,Cholesterol,Alk_Phos,Age
0,1,1,0.355556,0.238806,1,0.085196,0.105279,0.622822
1,1,0,0.177778,0.813433,1,0.109970,0.523509,0.578364
2,0,0,0.333333,0.567164,0,0.033837,0.016724,0.839597
3,1,0,0.144444,0.216418,1,0.074924,0.429723,0.545664
4,1,0,0.211111,0.585821,1,0.096073,0.028143,0.226748
...,...,...,...,...,...,...,...,...
407,1,0,0.211111,0.373134,0,0.157885,0.081122,0.780705
408,1,0,0.244444,0.697761,0,0.125982,0.089491,0.243911
409,0,0,0.100000,0.544776,0,0.125680,0.056345,0.588967
410,0,0,0.155556,0.667910,0,0.083625,0.038656,0.608178


BPSO

In [23]:
def objective_function_rf(m):
      
    pso_filtered_data = ensemble_fs[ensemble_fs.columns[np.argwhere(m==1).flatten()]]
    X_train, X_test, y_train, y_test = train_test_split(pso_filtered_data, y, test_size = 0.20, random_state = 1)
    resample = SMOTE(random_state = 1)
    X_train, y_train = resample.fit_resample(X_train, y_train)
    
    model = RandomForestClassifier(random_state = 1)

    train_model = model.fit(X_train, y_train)
    
    #Accuracy score for test dataset
    pred_test = train_model.predict(X_test)
    accuracy = accuracy_score(y_test, pred_test)
    fscore = f1_score(y_test, pred_test)

    return accuracy, fscore

In [24]:
def objective_function_svm(m):

    pso_filtered_data = ensemble_fs[ensemble_fs.columns[np.argwhere(m==1).flatten()]]
    X_train, X_test, y_train, y_test = train_test_split(pso_filtered_data, y, test_size = 0.20, random_state = 1)
    resample = SMOTE(random_state = 1)
    X_train, y_train = resample.fit_resample(X_train, y_train)
    
    model = SVC(random_state = 1)

    train_model = model.fit(X_train, y_train)
    
    #Accuracy score for test dataset
    pred_test = train_model.predict(X_test)
    accuracy = accuracy_score(y_test, pred_test)
    fscore = f1_score(y_test, pred_test)
    
    return accuracy, fscore

In [25]:
def objective_function_xgb(m):
      
    pso_filtered_data = ensemble_fs[ensemble_fs.columns[np.argwhere(m==1).flatten()]]
    X_train, X_test, y_train, y_test = train_test_split(pso_filtered_data, y, test_size = 0.20, random_state = 1)
    resample = SMOTE(random_state = 1)
    X_train, y_train = resample.fit_resample(X_train, y_train)

    model = XGBClassifier(random_state = 1)

    train_model = model.fit(X_train, y_train)
    
    #Accuracy score for test dataset
    pred_test = train_model.predict(X_test)
    accuracy = accuracy_score(y_test, pred_test)
    fscore = f1_score(y_test, pred_test)

    return accuracy, fscore

In [26]:
class BPSO:
    def __init__(self, ob: int, num_of_population: int, num_of_features: int, max_features: int, w: float, c1: float, c2: float) -> None:
        self.particle_positions = np.random.randint(0, 2, (num_of_population, num_of_features))
        # when no feature, generate again
        while (len(idxs := np.where(self.particle_positions.sum(1) == 0)[0]) > 0):
            for idx in idxs:
                self.particle_positions[idx] = np.random.randint(0, 2)
        # when exceed max number of feature, randomly choose one selected feature become 0
        for idx in np.where(self.particle_positions.sum(1) > max_features)[0]:
            while self.particle_positions[idx].sum() > max_features:
                one_idxs = np.where(self.particle_positions[idx] == 1)[0]
                self.particle_positions[idx][np.random.choice(one_idxs)] = 0

        self.particle_best_positions = self.particle_positions.copy()
        self.particle_best_scores = np.zeros(num_of_population, dtype=np.float32)
        self.particle_best_scores_2 = np.zeros(num_of_population, dtype=np.float32)
        self.particle_velocities = np.random.uniform(0., 1., (num_of_population, num_of_features))
        self.global_particle_best_score = .0
        self.global_particle_best_score_2 = .0
        self.global_particle_best_positions = np.zeros(num_of_features)
        self.ob = ob
        self.num_of_population = num_of_population
        self.max_features = max_features
        self.num_of_features = num_of_features
        self.w = w
        self.c1 = c1
        self.c2 = c2

        print("Done Initialization")

    def eval_all_particles(self) -> None:
        # update based on different objective function
        for i, particle in enumerate(self.particle_positions):
            
            if self.ob == 0:
                arr, f1 = objective_function_rf(particle)

            elif self.ob == 1:
                arr, f1 = objective_function_svm(particle)

            else:
                arr, f1 = objective_function_xgb(particle)

            if arr > self.particle_best_scores[i]: # if larger or equal: exploration increase (update position even same high score)
                self.particle_best_scores[i] = arr
                self.particle_best_scores_2[i] = f1
                self.particle_best_positions[i] = particle
            elif arr == self.particle_best_scores[i]:
                if f1 > self.particle_best_scores[i]:
                    self.particle_best_scores[i] = arr
                    self.particle_best_scores_2[i] = f1
                    self.particle_best_positions[i] = particle
                    
            if arr > self.global_particle_best_score:
                self.global_particle_best_positions = self.particle_positions[i].copy()
                self.global_particle_best_score = arr
                self.global_particle_best_score_2 = f1
            elif arr == self.global_particle_best_score:
                if f1 > self.global_particle_best_score_2:
                    self.global_particle_best_positions = self.particle_positions[i].copy()
                    self.global_particle_best_score = arr
                    self.global_particle_best_score_2 = f1

    def update_particles(self) -> None:
        # vectorization, instead updating one by one
        self.particle_velocities = self.w * self.particle_velocities + \
                                   self.c1 * np.random.uniform(0., 1., (self.num_of_population, self.num_of_features))  * (self.particle_best_positions - self.particle_positions) + \
                                   self.c2 * np.random.uniform(0., 1., (self.num_of_population, self.num_of_features)) * (self.global_particle_best_positions - self.particle_positions)
        self.particle_positions = (self.sigmoid(self.particle_velocities) > np.random.uniform(0., 1.)) * 1
        # make sure the features subset contains at least one features and not exceed the max features
        while (len(idxs := np.where(self.particle_positions.sum(1) == 0)[0]) > 0):
            for idx in idxs:
                self.particle_positions[idx] = np.random.randint(0, 2)
        for idx in np.where(self.particle_positions.sum(1) > self.max_features)[0]:
            while self.particle_positions[idx].sum() > self.max_features:
                one_idxs = np.where(self.particle_positions[idx] == 1)[0]
                self.particle_positions[idx][np.random.choice(one_idxs)] = 0

    def sigmoid(self, x: np.ndarray) -> np.ndarray:
        return 1. / (1. + np.exp(-x))

    def fit(self, iterations: int) -> Tuple[np.ndarray, np.ndarray, float]:
        assert iterations > 0
        for iteration in range(iterations):
            self.eval_all_particles()
            self.update_particles()
            print(f"Iteration: {iteration + 1}, Current best accuracy: "
                  f"{self.global_particle_best_score}, F1-Score: "
                  f"{self.global_particle_best_score_2}, Current best solution: "
                  f"{self.global_particle_best_positions}")
        return self.global_particle_best_positions, self.global_particle_best_score

In [30]:
# random forest
bpso = BPSO(0, 30, num_of_union_feature, num_feat, 0.7, 1.5, 2.5)
best_po_rf, best_score_rf = bpso.fit(30)

Done Initialization
Iteration: 1, Current best accuracy: 0.7710843373493976, F1-Score: 0.7323943661971831, Current best solution: [1 0 0 0 0 1 1 0]
Iteration: 2, Current best accuracy: 0.7710843373493976, F1-Score: 0.7323943661971831, Current best solution: [1 0 0 0 0 1 1 0]
Iteration: 3, Current best accuracy: 0.7710843373493976, F1-Score: 0.7323943661971831, Current best solution: [1 0 0 0 0 1 1 0]
Iteration: 4, Current best accuracy: 0.7710843373493976, F1-Score: 0.7323943661971831, Current best solution: [1 0 0 0 0 1 1 0]
Iteration: 5, Current best accuracy: 0.7710843373493976, F1-Score: 0.7323943661971831, Current best solution: [1 0 0 0 0 1 1 0]
Iteration: 6, Current best accuracy: 0.7710843373493976, F1-Score: 0.7323943661971831, Current best solution: [1 0 0 0 0 1 1 0]
Iteration: 7, Current best accuracy: 0.7710843373493976, F1-Score: 0.7323943661971831, Current best solution: [1 0 0 0 0 1 1 0]
Iteration: 8, Current best accuracy: 0.7710843373493976, F1-Score: 0.732394366197183

In [45]:
# svm
bpso = BPSO(1, 30, num_of_union_feature, num_feat, 0.7, 1.5, 2.5)
best_po_svm, best_score_svm = bpso.fit(30)

Done Initialization
Iteration: 1, Current best accuracy: 0.8192771084337349, F1-Score: 0.7692307692307692, Current best solution: [0 0 1 0 0 1 0 1]
Iteration: 2, Current best accuracy: 0.8192771084337349, F1-Score: 0.7692307692307692, Current best solution: [0 0 1 0 0 1 0 1]
Iteration: 3, Current best accuracy: 0.8192771084337349, F1-Score: 0.7692307692307692, Current best solution: [0 0 1 0 0 1 0 1]
Iteration: 4, Current best accuracy: 0.8192771084337349, F1-Score: 0.7692307692307692, Current best solution: [0 0 1 0 0 1 0 1]
Iteration: 5, Current best accuracy: 0.8192771084337349, F1-Score: 0.7692307692307692, Current best solution: [0 0 1 0 0 1 0 1]
Iteration: 6, Current best accuracy: 0.8192771084337349, F1-Score: 0.7692307692307692, Current best solution: [0 0 1 0 0 1 0 1]
Iteration: 7, Current best accuracy: 0.8192771084337349, F1-Score: 0.7692307692307692, Current best solution: [0 0 1 0 0 1 0 1]
Iteration: 8, Current best accuracy: 0.8192771084337349, F1-Score: 0.769230769230769

In [32]:
# xgboost
bpso = BPSO(2, 30, num_of_union_feature, num_feat, 0.7, 1.5, 2.5)
best_po_xgb, best_score_xgb = bpso.fit(30)

Done Initialization
Iteration: 1, Current best accuracy: 0.7228915662650602, F1-Score: 0.684931506849315, Current best solution: [1 0 1 0 0 0 1 0]
Iteration: 2, Current best accuracy: 0.7469879518072289, F1-Score: 0.6956521739130435, Current best solution: [1 0 0 0 0 1 1 0]
Iteration: 3, Current best accuracy: 0.7469879518072289, F1-Score: 0.6956521739130435, Current best solution: [1 0 0 0 0 1 1 0]
Iteration: 4, Current best accuracy: 0.7469879518072289, F1-Score: 0.6956521739130435, Current best solution: [1 0 0 0 0 1 1 0]
Iteration: 5, Current best accuracy: 0.7469879518072289, F1-Score: 0.6956521739130435, Current best solution: [1 0 0 0 0 1 1 0]
Iteration: 6, Current best accuracy: 0.7469879518072289, F1-Score: 0.6956521739130435, Current best solution: [1 0 0 0 0 1 1 0]
Iteration: 7, Current best accuracy: 0.7469879518072289, F1-Score: 0.6956521739130435, Current best solution: [1 0 0 0 0 1 1 0]
Iteration: 8, Current best accuracy: 0.7469879518072289, F1-Score: 0.6956521739130435

<b>Classification with feature selection</b>

Random Forest

In [47]:
# random forest
pso_selected_columns_rf = np.argwhere(best_po_rf==1).flatten()
pso_filtered_data_rf = ensemble_fs[ensemble_fs.columns[pso_selected_columns_rf]]
pso_filtered_data_rf

,Hepatomegaly,Cholesterol,Alk_Phos
0,1,0.085196,0.105279
1,1,0.109970,0.523509
2,0,0.033837,0.016724
3,1,0.074924,0.429723
4,1,0.096073,0.028143
...,...,...,...
407,1,0.157885,0.081122
408,1,0.125982,0.089491
409,0,0.125680,0.056345
410,0,0.083625,0.038656


In [48]:
X_train, X_test, y_train, y_test = train_test_split(pso_filtered_data_rf, y, test_size = 0.20, random_state = 1)
resample = SMOTE(random_state = 1)
X_train, y_train = resample.fit_resample(X_train, y_train)

model = RandomForestClassifier(random_state = 1)
train_model = model.fit(X_train, y_train)
pred_test = train_model.predict(X_test)

print("Accuracy: {}".format((accuracy_score(pred_test, y_test)*100).round(2)))
print("Precision: {}".format((precision_score(pred_test, y_test)*100).round(2)))
print("Recall: {}".format((recall_score(pred_test, y_test)*100).round(2)))
print("F1-score: {}".format((f1_score(pred_test, y_test)*100).round(2)))
print("Confusion matrix: ", confusion_matrix(pred_test, y_test))

Accuracy: 77.11
Precision: 81.25
Recall: 66.67
F1-score: 73.24
Confusion matrix:  [[38  6]
 [13 26]]


Support Vector Machine

In [49]:
# svm
pso_selected_columns_svm = np.argwhere(best_po_svm==1).flatten()
pso_filtered_data_svm = ensemble_fs[ensemble_fs.columns[pso_selected_columns_svm]]
pso_filtered_data_svm

,Prothrombin,Cholesterol,Age
0,0.355556,0.085196,0.622822
1,0.177778,0.109970,0.578364
2,0.333333,0.033837,0.839597
3,0.144444,0.074924,0.545664
4,0.211111,0.096073,0.226748
...,...,...,...
407,0.211111,0.157885,0.780705
408,0.244444,0.125982,0.243911
409,0.100000,0.125680,0.588967
410,0.155556,0.083625,0.608178


In [50]:

X_train, X_test, y_train, y_test = train_test_split(pso_filtered_data_svm, y, test_size = 0.20, random_state = 1)
resample = SMOTE(random_state = 1)
X_train, y_train = resample.fit_resample(X_train, y_train)

model = SVC(random_state = 1)
train_model = model.fit(X_train, y_train)
pred_test = train_model.predict(X_test)

print("Accuracy: {}".format((accuracy_score(pred_test, y_test)*100).round(2)))
print("Precision: {}".format((precision_score(pred_test, y_test)*100).round(2)))
print("Recall: {}".format((recall_score(pred_test, y_test)*100).round(2)))
print("F1-score: {}".format((f1_score(pred_test, y_test)*100).round(2)))
print("Confusion matrix: ", confusion_matrix(pred_test, y_test))

Accuracy: 81.93
Precision: 78.12
Recall: 75.76
F1-score: 76.92
Confusion matrix:  [[43  7]
 [ 8 25]]


XGboost

In [51]:
# xgboost
pso_selected_columns_xgb = np.argwhere(best_po_xgb==1).flatten()
pso_filtered_data_xgb = ensemble_fs[ensemble_fs.columns[pso_selected_columns_xgb]]
pso_filtered_data_xgb

,Hepatomegaly,Cholesterol,Alk_Phos
0,1,0.085196,0.105279
1,1,0.109970,0.523509
2,0,0.033837,0.016724
3,1,0.074924,0.429723
4,1,0.096073,0.028143
...,...,...,...
407,1,0.157885,0.081122
408,1,0.125982,0.089491
409,0,0.125680,0.056345
410,0,0.083625,0.038656


In [52]:
X_train, X_test, y_train, y_test = train_test_split(pso_filtered_data_xgb, y, test_size = 0.20, random_state = 1)
resample = SMOTE(random_state = 1)
X_train, y_train = resample.fit_resample(X_train, y_train)

model = XGBClassifier(random_state = 1)
train_model = model.fit(X_train, y_train)
pred_test = train_model.predict(X_test)

print("Accuracy: {}".format((accuracy_score(pred_test, y_test)*100).round(2)))
print("Precision: {}".format((precision_score(pred_test, y_test)*100).round(2)))
print("Recall: {}".format((recall_score(pred_test, y_test)*100).round(2)))
print("F1-score: {}".format((f1_score(pred_test, y_test)*100).round(2)))
print("Confusion matrix: ", confusion_matrix(pred_test, y_test))

Accuracy: 74.7
Precision: 75.0
Recall: 64.86
F1-score: 69.57
Confusion matrix:  [[38  8]
 [13 24]]
